In [81]:
import pandas as pd
from datetime import datetime

In [90]:
df1 = pd.read_csv('Data/GOES_16_merged_data.csv')
df2 = pd.read_csv('Data/GOES_17_merged_data.csv')
df3 = pd.read_csv('Data/GOES_18_merged_data.csv')

merged_df = pd.merge(df1, df2, on='time', how='outer')
merged_df = pd.merge(merged_df, df3, on='time', how='outer')
print(len(merged_df))

merged_df = merged_df[~merged_df['time'].duplicated(keep='first')]
print(len(merged_df))

/var/folders/r7/6cts6h556zzgbx26m_g0p4gw0000gn/T/ipykernel_36022/389087925.py:2: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Data/GOES_17_merged_data.csv')


3548372
3548160


In [91]:
duplicates = merged_df[merged_df.duplicated('time', keep=False)]

# Display duplicate rows
print("Duplicate Time Values:")
print(duplicates)
print(len(duplicates))
print(len(merged_df))
# If you just want to know if there are any duplicates
has_duplicates = merged_df['time'].duplicated().any()
print(f"Are there duplicate time values? {has_duplicates}")

Duplicate Time Values:
Empty DataFrame
Columns: [time, xrsa_flux_x, xrsb_flux_x, status_x, flare_class_x, xrsa_flux_y, xrsb_flux_y, status_y, flare_class_y, xrsa_flux, xrsb_flux, status, flare_class]
Index: []
0
3548160
Are there duplicate time values? False


In [92]:
first_value = merged_df.head(1)
first_value_time = str(first_value['time'].item())
first_value_time = datetime.strptime(first_value_time, '%Y-%m-%d %H:%M:%S')
print(first_value_time)

2017-02-07 00:00:00


In [93]:
last_value = merged_df.tail(1)
last_value_time = str(last_value['time'].item())
last_value_time = datetime.strptime(last_value_time, '%Y-%m-%d %H:%M:%S')
print(last_value_time)

2023-11-06 23:59:00


In [94]:
time_difference = last_value_time - first_value_time
amount_of_minutes = int(time_difference.total_seconds() / 60)
print(amount_of_minutes)

3548159


In [95]:
print(len(merged_df))
print()

3548160


In [96]:
merged_df.to_csv("full_set.csv", index=False)